<a href="https://colab.research.google.com/github/mathluva/Udemy-BERT/blob/main/Udemy_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dependencies
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
#use ! for terminal commands
!pip install bert-for-tf2 #tensorflow2 light version
!pip install sentencepiece #required for BERT-tf2

     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=883a64613921f7747e332cefc56abc2bde02031f1df5fa529bc8f5c8de306c42
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=3ba528c920d9c5cbc9ce077a0544dbc9f8f2a42abc18a867f863ce1882bd9277
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=3bca608ea86cfbefc09bb3bb2814e86d973d0f75bd6e007c28dfc3773e014271
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 9.1MB/s 


In [4]:
try:
    %tensorflow_version 2.x #only available in Google colab
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub #used to import the weights from BERT

from tensorflow.keras import layers
import bert #installed in previous step

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x #only available in Google colab`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


# Data preprocessing


In [5]:
#load files, data preprocessing
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
#label columns
#latin1 is common for western languages
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/trainingandtestdata.zip (Unzipped Files)/training.1600000.processed.noemoticon.csv", 
    header = None,
    names = cols,
    engine = "python",
    encoding = "latin1")

In [11]:
#axis1 column data
#without inplace=True, it would be required to write data = data.drop("...")
data.drop(["id", "date","query", "user"], axis = 1, inplace = True)

In [12]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [13]:
#cleaning
#r is regEX (regexr.com for more documentation)
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text() #tweets are not usuable as standard string, need BS to extract string
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ',tweet)#anything behind @symbol with empty space, apply to tweet
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)#? means the s can be there or not
    tweet = re.sub(r"[^a-zA-Z.!?]", ' ', tweet) #keep only standard characters
    tweet = re.sub(r" +", ' ', tweet) #take multiple sequences of white space with only one white space
    return tweet

In [15]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [16]:
#process sentiment
data_labels = data.sentiment.values
data_labels[data_labels ==4] =1 #data is using 0 and 4, replace 4 with standard 1

# Tokenization


In [ ]:
#create BERT layer to have access to metadata for the tokenizer(like vocab size).
#call BERT as a layer, hub is where all pretrained models are located
#trainable = False bc 
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en-uncased_L-12_H-768_A-12/1",
                            trainable = False) 